In [4]:
library(tidyverse)
library(targets)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [125]:
combined_pldf = qs::qread('_targets/objects/combined_pldf_obob5v5')
combined_pldf %>% head

,polar_label
,<chr>
TCAGGTAAGCACCTGC_1_1,a1_Tany__Astrocytes.neg
CTACAGACAACTTGCA_1_1,a1_Tany__Astrocytes.none
GCAGCTGAGTCACTAC_1_1,a1_Tany__Astrocytes.none
AGAGCAGGTCTGATAC_1_1,a1_Tany__Astrocytes.none
TTTGGTTTCCCTCTCC_1_1,a1_Tany__Astrocytes.none
TGTCCCAAGCATTGTC_1_1,a1_Tany__Astrocytes.none


In [126]:
split_clust_only = combined_pldf %>% 
tibble %>%
mutate(label = str_replace(polar_label, ".none", "")) %>%
mutate(label = str_replace(label, ".neg", "")) %>%
mutate(label = str_replace(label, ".pos", "")) %>%       
mutate(polarity = case_when(str_ends(polar_label, '.none') ~ "none",
                         str_ends(polar_label, '.pos') ~ "pos",
                         str_ends(polar_label, '.neg') ~ "neg")) %>%
distinct

split_clust_summary = split_clust_only %>%
select(-polar_label) %>% distinct %>%
group_by(label) %>%
summarise(n = n()) %>%
filter(n > 1) %>%
arrange(desc(n))

split_clust_summary

label,n
<chr>,<int>
a1_Tany__Astrocytes,3
Agrp,3
Astrocytes,3
b1_Tanycytes,3
Htr3b,3
MOL,3
NG2_OPC2,3
Sst_Unc13c__Agrp,3
Trh_Lef1,3


In [127]:
milo_ngo_tibble = tibble(label = split_clust_summary$label)
milo_ngo_tibble$milo_obj = milo_ngo_tibble %>%
    pull(label) %>%
    paste0("milo_", .) %>%
    rlang::syms()
milo_ngo_tibble$da_results_obj = milo_ngo_tibble %>%
    pull(label) %>%
    paste0("da_results_02_", ., "___obob5v5") %>%
    rlang::syms()
milo_ngo_tibble

label,milo_obj,da_results_obj
<chr>,<list>,<list>
a1_Tany__Astrocytes,milo_a1_Tany__Astrocytes,da_results_02_a1_Tany__Astrocytes___obob5v5
Agrp,milo_Agrp,da_results_02_Agrp___obob5v5
Astrocytes,milo_Astrocytes,da_results_02_Astrocytes___obob5v5
b1_Tanycytes,milo_b1_Tanycytes,da_results_02_b1_Tanycytes___obob5v5
Htr3b,milo_Htr3b,da_results_02_Htr3b___obob5v5
MOL,milo_MOL,da_results_02_MOL___obob5v5
NG2_OPC2,milo_NG2_OPC2,da_results_02_NG2_OPC2___obob5v5
Sst_Unc13c__Agrp,milo_Sst_Unc13c__Agrp,da_results_02_Sst_Unc13c__Agrp___obob5v5
Trh_Lef1,milo_Trh_Lef1,da_results_02_Trh_Lef1___obob5v5


In [128]:
qs::qsave(milo_ngo_tibble, 'milo_ngo_tibble.qs')

In [129]:
a = factor(c('neg', 'none', 'pos'), levels=c('pos', 'neg', 'none'))
a

[1] neg  none pos 
Levels: pos neg none

In [130]:
sort(a)

[1] pos  neg  none
Levels: pos neg none

In [131]:
# select only one pair of polarities to test for
# if all three present, select pos.neg
# otherwise selection is made already

polarity_comparison_tibble = split_clust_only %>%
filter(label %in% split_clust_summary$label) %>%
select(-polar_label) %>%
group_by(label) %>%
mutate(polarity = factor(polarity, levels=c('pos', 'neg', 'none'))) %>%
group_by(label) %>%
mutate(polarity_comparison = paste0(sort(polarity)[1:2], collapse='.')) %>%
select(-polarity) %>%
distinct %>% tibble
polarity_comparison_tibble

label,polarity_comparison
<chr>,<chr>
a1_Tany__Astrocytes,pos.neg
a2_Tanycytes,pos.none
Agrp,pos.neg
Astrocytes,pos.neg
b1_Tanycytes,pos.neg
Htr3b,pos.neg
MOL,pos.neg
NG2_OPC2,pos.neg
Sst_Pthlh.sc23,pos.none


In [132]:
milo_ngo_tibble = left_join(milo_ngo_tibble, polarity_comparison_tibble, by = 'label')
milo_ngo_tibble

label,milo_obj,da_results_obj,polarity_comparison
<chr>,<list>,<list>,<chr>
a1_Tany__Astrocytes,milo_a1_Tany__Astrocytes,da_results_02_a1_Tany__Astrocytes___obob5v5,pos.neg
Agrp,milo_Agrp,da_results_02_Agrp___obob5v5,pos.neg
Astrocytes,milo_Astrocytes,da_results_02_Astrocytes___obob5v5,pos.neg
b1_Tanycytes,milo_b1_Tanycytes,da_results_02_b1_Tanycytes___obob5v5,pos.neg
Htr3b,milo_Htr3b,da_results_02_Htr3b___obob5v5,pos.neg
MOL,milo_MOL,da_results_02_MOL___obob5v5,pos.neg
NG2_OPC2,milo_NG2_OPC2,da_results_02_NG2_OPC2___obob5v5,pos.neg
Sst_Unc13c__Agrp,milo_Sst_Unc13c__Agrp,da_results_02_Sst_Unc13c__Agrp___obob5v5,pos.neg
Trh_Lef1,milo_Trh_Lef1,da_results_02_Trh_Lef1___obob5v5,pos.neg


In [133]:
qs::qsave(milo_ngo_tibble, 'milo_ngo_tibble.qs')

In [140]:
milo_ngo_tibble %>% pull(polarity_comparison) %>% str_split(pattern=fixed('.')) %>% unlist()

[1] "pos"  "neg"  "pos"  "neg"  "pos"  "neg"  "pos"  "neg"  "pos"  "neg" 
[11] "pos"  "neg"  "pos"  "neg"  "pos"  "neg"  "pos"  "neg"  "pos"  "none"
[21] "pos"  "none" "pos"  "none"

In [139]:
str_split('pos.neg', pattern=fixed('.')) %>% unlist

[1] "pos" "neg"